# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 27 2023 12:28PM,258246,19,ADV80012894,"AdvaGen Pharma, Ltd",Released
1,Feb 27 2023 12:22PM,258245,12,7648410,Hush Hush,Released
2,Feb 27 2023 12:13PM,258244,19,9213195,VITABIOTICS USA INC.,Released
3,Feb 27 2023 12:13PM,258244,19,9213197,VITABIOTICS USA INC.,Released
4,Feb 27 2023 12:13PM,258244,19,9213198,VITABIOTICS USA INC.,Released
5,Feb 27 2023 11:57AM,258234,21,RHL022423,"OK Capsule, Inc.",Released
6,Feb 27 2023 11:56AM,258233,10,DNMD-35148,Emerginnova,Released
7,Feb 27 2023 11:56AM,258233,10,Enova-11418,Emerginnova,Released
8,Feb 27 2023 11:54AM,258231,10,SO94550,"Senseonics, Incorporated",Released
9,Feb 27 2023 11:48AM,258227,10,9212681,"StellaLife, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,258233,Released,2
33,258234,Released,1
34,258244,Released,3
35,258245,Released,1
36,258246,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258233,NaN,2.0
258234,NaN,1.0
258244,NaN,3.0
258245,NaN,1.0
258246,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258149,0.0,18.0
258153,1.0,30.0
258162,0.0,58.0
258166,17.0,0.0
258188,0.0,24.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258149,0,18
258153,1,30
258162,0,58
258166,17,0
258188,0,24


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258149,0,18
1,258153,1,30
2,258162,0,58
3,258166,17,0
4,258188,0,24


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258149,,18
1,258153,1,30
2,258162,,58
3,258166,17,
4,258188,,24


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 27 2023 12:28PM,258246,19,"AdvaGen Pharma, Ltd"
1,Feb 27 2023 12:22PM,258245,12,Hush Hush
2,Feb 27 2023 12:13PM,258244,19,VITABIOTICS USA INC.
5,Feb 27 2023 11:57AM,258234,21,"OK Capsule, Inc."
6,Feb 27 2023 11:56AM,258233,10,Emerginnova
8,Feb 27 2023 11:54AM,258231,10,"Senseonics, Incorporated"
9,Feb 27 2023 11:48AM,258227,10,"StellaLife, Inc."
10,Feb 27 2023 11:37AM,258219,10,ISDIN Corporation
71,Feb 27 2023 11:26AM,258221,19,"Innogenix, LLC"
96,Feb 27 2023 11:23AM,258220,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 27 2023 12:28PM,258246,19,"AdvaGen Pharma, Ltd",,1
1,Feb 27 2023 12:22PM,258245,12,Hush Hush,,1
2,Feb 27 2023 12:13PM,258244,19,VITABIOTICS USA INC.,,3
3,Feb 27 2023 11:57AM,258234,21,"OK Capsule, Inc.",,1
4,Feb 27 2023 11:56AM,258233,10,Emerginnova,,2
5,Feb 27 2023 11:54AM,258231,10,"Senseonics, Incorporated",,1
6,Feb 27 2023 11:48AM,258227,10,"StellaLife, Inc.",,1
7,Feb 27 2023 11:37AM,258219,10,ISDIN Corporation,,61
8,Feb 27 2023 11:26AM,258221,19,"Innogenix, LLC",,25
9,Feb 27 2023 11:23AM,258220,10,Eywa Pharma Inc.,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 27 2023 12:28PM,258246,19,"AdvaGen Pharma, Ltd",1,
1,Feb 27 2023 12:22PM,258245,12,Hush Hush,1,
2,Feb 27 2023 12:13PM,258244,19,VITABIOTICS USA INC.,3,
3,Feb 27 2023 11:57AM,258234,21,"OK Capsule, Inc.",1,
4,Feb 27 2023 11:56AM,258233,10,Emerginnova,2,
5,Feb 27 2023 11:54AM,258231,10,"Senseonics, Incorporated",1,
6,Feb 27 2023 11:48AM,258227,10,"StellaLife, Inc.",1,
7,Feb 27 2023 11:37AM,258219,10,ISDIN Corporation,61,
8,Feb 27 2023 11:26AM,258221,19,"Innogenix, LLC",25,
9,Feb 27 2023 11:23AM,258220,10,Eywa Pharma Inc.,5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 27 2023 12:28PM,258246,19,"AdvaGen Pharma, Ltd",1,
1,Feb 27 2023 12:22PM,258245,12,Hush Hush,1,
2,Feb 27 2023 12:13PM,258244,19,VITABIOTICS USA INC.,3,
3,Feb 27 2023 11:57AM,258234,21,"OK Capsule, Inc.",1,
4,Feb 27 2023 11:56AM,258233,10,Emerginnova,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 27 2023 12:28PM,258246,19,"AdvaGen Pharma, Ltd",1.0,NaN
1,Feb 27 2023 12:22PM,258245,12,Hush Hush,1.0,NaN
2,Feb 27 2023 12:13PM,258244,19,VITABIOTICS USA INC.,3.0,NaN
3,Feb 27 2023 11:57AM,258234,21,"OK Capsule, Inc.",1.0,NaN
4,Feb 27 2023 11:56AM,258233,10,Emerginnova,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 27 2023 12:28PM,258246,19,"AdvaGen Pharma, Ltd",1.0,0.0
1,Feb 27 2023 12:22PM,258245,12,Hush Hush,1.0,0.0
2,Feb 27 2023 12:13PM,258244,19,VITABIOTICS USA INC.,3.0,0.0
3,Feb 27 2023 11:57AM,258234,21,"OK Capsule, Inc.",1.0,0.0
4,Feb 27 2023 11:56AM,258233,10,Emerginnova,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4647655,350.0,6.0
12,258245,1.0,0.0
15,774611,59.0,85.0
16,258201,1.0,0.0
19,2065711,33.0,17.0
20,258196,0.0,1.0
21,258234,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4647655,350.0,6.0
1,12,258245,1.0,0.0
2,15,774611,59.0,85.0
3,16,258201,1.0,0.0
4,19,2065711,33.0,17.0
5,20,258196,0.0,1.0
6,21,258234,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,350.0,6.0
1,12,1.0,0.0
2,15,59.0,85.0
3,16,1.0,0.0
4,19,33.0,17.0
5,20,0.0,1.0
6,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,350.0
1,12,Released,1.0
2,15,Released,59.0
3,16,Released,1.0
4,19,Released,33.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,6.0,0.0,85.0,0.0,17.0,1.0,0.0
Released,350.0,1.0,59.0,1.0,33.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,6.0,0.0,85.0,0.0,17.0,1.0,0.0
1,Released,350.0,1.0,59.0,1.0,33.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,6.0,0.0,85.0,0.0,17.0,1.0,0.0
1,Released,350.0,1.0,59.0,1.0,33.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()